In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pyblp as blp
import torch
from torch.autograd import Variable
import torch.optim as optim
from linearmodels.iv import IV2SLS

blp.options.digits = 2
blp.options.verbose = False
nax = np.newaxis

## Exercise 3

The file `ps1_ex3.csv` contains aggregate data on a large number $T=1000$ of markets in which $J=6$ products compete between each other together with an outside good $j=0$. The utility of consumer $i$ is given by:

$$
\begin{aligned}
&u_{i j t}=-\alpha p_{j t}+\beta x_{j t}+\xi_{j t}+\epsilon_{i j t} \quad j=1, \ldots, 6 \\
&u_{i 0 t}=\epsilon_{i 0 t}
\end{aligned}
$$

where $p_{j t}$ is the price of product $j$ in market $t, x_{j t}$ is an observed product characteristic, $\xi_{j t}$ is an unobserved product characteristic and $\epsilon_{i j t}$ is i.i.d T1EV $(0,1)$. Our goal is to to estimate demand parameters $(\alpha, \beta)$ and perform some counterfactual exercises.

In [2]:
# Load the dataset.
data_ex3 = pd.read_csv('ps1_ex3.csv')
num_prod = data_ex3.Product.max()
num_T = data_ex3.market.max()

### Part A

Assuming that the variable $z$ in the dataset is a valid instrument for prices, write down the moment condition that allows you to consistently estimate $(\alpha, \beta)$ and obtain an estimate for both parameters.

------------------------------------------------------------------------------------------------------

Under the T1EV assumption, we can derive the the con which corresponds to the predicted market share for product $j$ at time $t$. This can be approximated from the data using the observed market share $s_{jt}$.

$$
\operatorname{Pr}(i \text{ chooses }j \text{ at time } t) \; = \; \frac{\exp \left(-\alpha p_{jt}+{x}_{jt} \beta + \xi_{jt}\right)}{\sum_{k \in \mathcal{J}_{t}} \exp \left(-\alpha p_{kt}+{x}_{kt} \beta+\xi_{kt}\right)} \; \approx \; s_{jt}
$$

We can invoke the normalization assumption on $u_{i0}$ and take the logarithm of the share ratio $s_{jt}/s_{0t}$ to obtain

$$
    \ln \left({\frac{s_{jt}}{s_{0t}}\right) \; = \; -\alpha p_{jt}+ {x}_{jt} \beta+\xi_{jt}
$$

In [3]:
# Create outside option shares and merge into dataset.

share_total = data_ex3.groupby(['market'])['Shares'].sum().reset_index()
share_total.rename(columns={'Shares': 's0'}, inplace=True)
share_total['s0'] = 1 - share_total['s0']
data_ex3 = pd.merge(data_ex3, share_total, on='market')

In [4]:
# Create natural log of share ratios

data_ex3['s_ratio'] = np.log(data_ex3['Shares']/data_ex3['s0'])

Given that $z_{jt}$ is a relevant instrument for $p_{jt}$ and that $x_{jt}$ is exogenous, we can impose the conditional exogeneity restriction
 $$
    \mathbb{E}\left[\xi_{jt} \mid x_{jt}, z_{jt} \right] = 0
 $$
  in order to estimate $\alpha$ and $\beta$. Using the Law of Iterated Expectations, we can conclude that

$$
    \mathbb{E} \left[ \begin{pmatrix} x_{jt} \\ z_{jt} \end{pmatrix} \xi_{jt} \right] \; = \;  \mathbb{E} \left[ \begin{pmatrix} x_{jt} \\ z_{jt} \end{pmatrix} \left\{\ln \left({\frac{s_{jt}}{s_{0t}}\right) + \alpha p_{jt} - {x}_{jt} \beta \right\}\right] \; = \; \begin{pmatrix} 0 \\ 0 \end{pmatrix}
$$

Given that we have $6$ products and $2$ moment conditions for each product, we are over-identifying $\alpha$ and $\beta$.

GMM provides the minimizer corresponding to a quadratic loss function with 12 moments.

$$
    \begin{pmatrix} \widehat{\alpha} \\ \widehat{\beta} \end{pmatrix} \quad \in \;\; \underset{\begin{pmatrix} \alpha \\ \beta \end{pmatrix}}{\arg \min}  \left[ \frac{1}{T} \sum_{t} x_{jt} \left\{\ln \left({\frac{s_{jt}}{s_{0t}}\right) + \alpha p_{jt} - {x}_{jt} \beta \right\} \right]
$$

I perform the two-step procedure to obtain the efficient GMM estimator of the model parameters.


In [5]:
# Set parameters for the GMM optimization.
α = Variable(torch.ones(1), requires_grad=True)
β = Variable(torch.ones(1), requires_grad=True)

lns = torch.tensor(data_ex3['s_ratio'])
p = torch.tensor(data_ex3['Prices'])
x = torch.tensor(data_ex3['x'])
z = torch.tensor(data_ex3['z'])

weight_matrix = torch.eye(num_prod * 2, dtype=torch.double)

def ex3_gmm_eqns(a, b):

    cond1 = (lns + a * p - b * x) * x
    cond2 = (lns + a * p - b * x) * z

    return [cond1.reshape((num_T, num_prod)),
            cond2.reshape((num_T, num_prod))]

# Compile the moments required for GMM.
def gmm_moments(a, b):

    moments = [eq.mean(axis=0) for eq in ex3_gmm_eqns(a, b)]
    return torch.cat(tuple(moments), 0)

# Define the objective function for GMM.
def gmm_loss(a, b):

    moments = gmm_moments(a, b)
    return moments[None, :] @ weight_matrix @ moments[:, None]

# Define the Jacobian of moments vector G.
def return_g(a, b):

    g = torch.autograd.functional.jacobian(gmm_moments, (a, b))
    return torch.cat(g, 1)

# Define the variance of the moments.
def return_var_g(a, b):

    g = torch.cat(tuple(ex3_gmm_eqns(a, b)), 1)
    return 1/num_T * (g.T @ g)

# Define the adaptive gradient descent optimizer used to find the estimates.
opt_gmm = optim.Adam([α, β], lr=0.05)

In [6]:
# Optimizing over the GMM loss function
for epoch in range(10000):

    opt_gmm.zero_grad() # Reset gradient inside the optimizer

    # Compute the objective at the current parameter values.
    loss_round1 = gmm_loss(a=α, b=β)
    loss_round1.backward() # Gradient computed.
    opt_gmm.step()     # Update parameter values using gradient descent.

In [7]:
α, β

(tensor([0.2328], requires_grad=True), tensor([0.2889], requires_grad=True))

In [8]:
# Set parameters for second round of the GMM optimization.
α2 = Variable(torch.ones(1), requires_grad=True)
β2 = Variable(torch.ones(1), requires_grad=True)

# Construct optimal weight matrix from Step 1.
weight_matrix = torch.inverse(return_var_g(α, β)).detach()

# Create new optimization object for Step 2 of GMM.
opt_gmm2 = optim.Adam([α2, β2], lr=0.05)

In [9]:
# Optimizing over the GMM loss function
for epoch in range(10000):

    opt_gmm2.zero_grad() # Reset gradient inside the optimizer

    # Compute the objective at the current parameter values.
    loss_round2 = gmm_loss(a=α2, b=β2)
    loss_round2.backward() # Gradient computed.
    opt_gmm2.step()      # Update parameter values using gradient descent.

In [10]:
G = return_g(α2, β2).double()
V = return_var_g(α2, β2).double()

V2 = torch.inverse(G.T @ torch.inverse(V) @ G)
V2 = V2.detach()/num_T
torch.sqrt(torch.diag(V2))

tensor([0.0014, 0.0059], dtype=torch.float64)

We find the following estimates for $\alpha$ and $\beta$.

|          | Estimate | Std. Error |
|:--------:|:--------:|:----------:|
| $\alpha$ |  0.2342  |   0.0014   |
| $\beta$  |  0.2935  |   0.0059   |

We can verify the procedure by comparing the results from running 2SLS on the same sample.


In [11]:
iv = IV2SLS(dependent=data_ex3['s_ratio'],
       exog=data_ex3['x'],
       endog=data_ex3['Prices'],
       instruments=data_ex3['z']).fit(cov_type='unadjusted')

print(iv.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                s_ratio   R-squared:                      0.7066
Estimator:                    IV-2SLS   Adj. R-squared:                 0.7065
No. Observations:                6000   F-statistic:                 1.338e+04
Date:                Wed, Jan 26 2022   P-value (F-stat)                0.0000
Time:                        13:30:11   Distribution:                  chi2(2)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
x              0.2876     0.0065     44.586     0.0000      0.2749      0.3002
Prices        -0.2396     0.0022    -108.04     0.00

### Part B

We know that the elasticities for homogenous demand are given by

$$
    \varepsilon_{j k, t} \; = \;
    \begin{cases}-\alpha p_{j, t}\left(1-\pi_{j, t}\right) & \text { if } j=k \\ \alpha p_{k, t} \pi_{k, t} & \text { otherwise }\end{cases}
$$

In [12]:
α, β = α2.item(), β2.item()

In [13]:
# Compute the own and cross-price elasticity for each market and pair of products.
data_ex3['own'] = -α * data_ex3['Prices'] * (1 - data_ex3['Shares'])
data_ex3['cross'] = α * data_ex3['Prices'] * data_ex3['Shares']
e_mean = data_ex3.groupby(['Product'])[['own', 'cross']].mean()

# Generate matrix of average elasticities.
e_mat = np.tile(e_mean['cross'], (num_prod, 1))
np.fill_diagonal(e_mat, e_mean['own'])

# Convert it to a dataframe.
prod_list = list(map(str, range(1, num_prod+ 1)))
e_mat = pd.DataFrame(e_mat, index=prod_list, columns=prod_list)

In [14]:
e_mat

,1,2,3,4,5,6
1,-0.626071,0.161827,0.064499,0.063705,0.062882,0.064863
2,0.160862,-0.626882,0.064499,0.063705,0.062882,0.064863
3,0.160862,0.161827,-0.645872,0.063705,0.062882,0.064863
4,0.160862,0.161827,0.064499,-0.648239,0.062882,0.064863
5,0.160862,0.161827,0.064499,0.063705,-0.647014,0.064863
6,0.160862,0.161827,0.064499,0.063705,0.062882,-0.646700


The results above show that the elasticity 